In [1]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import time
print(tf.__version__)

start = time.time()
with tf.Session( ) as sess:
    # deserialize the frozen graph
#     with tf.gfile.GFile("./facenet/facenet.pb", "rb") as f:
    with tf.gfile.GFile("./pb/fn_vgg2.pb", "rb") as f:
        frozen_graph = tf.GraphDef()
        frozen_graph.ParseFromString(f.read())
end = time.time()
print(end-start)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

1.15.5
5.70016884803772


In [3]:
# for node in frozen_graph.node:¶
    # print(node.name)
node_num = len([1 for n in frozen_graph.node] )
print(node_num)

2280


In [5]:
start = time.time()
# output_nodes = ["embeddings"]
# input_node   = "input"

output_nodes = ["Bottleneck_BatchNorm/batchnorm_1/add_1:0"]
calib_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_nodes,
    max_batch_size=1,
    max_workspace_size_bytes= 1<<27,
    precision_mode='FP16',
    minimum_segment_size=3,
)


print(str(time.time()-start))

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Running against TensorRT version 7.1.3
87.69304156303406


In [6]:
# trt_engine_ops = len([1 for n in calib_graph.node if str(n.op)=='TRTEngineOp'])
trt_engine_ops = len([1 for n in calib_graph.node] )
print(trt_engine_ops)

4


In [7]:
# from tensorflow.python.compiler.tensorrt import trt_convert

# converter = trt_convert.TrtGraphConverter(input_graph_def=frozen_graph,nodes_blacklist=output_nodes)
# trt_graph = converter.convert()

# with open('./trt/fn_vgg_fp16.pb', 'wb') as f:
#     f.write(trt_graph.SerializeToString())

In [8]:
tf.train.write_graph(calib_graph, "pb", "fn_fp16_64MB.pb", as_text=False)

'pb/fn_fp16_64MB.pb'

In [4]:
1<<26

67108864